# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía - Simulación   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   

En este notebook se desarrolla el modelo teórico propuesto en el trabajo para la evaluación de armónicos esféricos mediante la utilización de computación cuántica. Se utiliza aquí un procesador cuántico real para realizar el experimento. El resultado se guarda en un el archivo especificado en "strNombreArchivo" y se permite la reanudación del experimento donde se quedó.

Primeramente, es necesario instalar la librería "pennylane", que será utilizada para los circuitos cuánticos.

In [1]:
%pip install pennylane > o
%pip install pennylane-qiskit > o

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
strNombreArchivo="results.csv"



Importación de librerías utilizadas en este archivo.

In [3]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm
import csv

Ángulos utilizados para el cálculo de los armónicos esféricos. La primera coordenada corresponde con la coordenada azimutal y la segunda con la polar. Estos ángulos se corresponden con la posición de 64 electrodos de un EEG utilizando el "sistema 10-10" para elegir la posición de los mismos, considerando la cabeza como una esfera.

In [4]:
angles = [['Fc5.',-71,-21],
 ['Fc3.',-50,-28],
 ['Fc1.',-32,-45],
 ['Fcz.',23,90],
 ['Fc2.',32,45],
 ['Fc4.',50,28],
 ['Fc6.',71,21],
 ['C5..',-69,0],
 ['C3..',-46,0],
 ['C1..',23,0],
 ['Cz..',0,0],
 ['C2..',23,0],
 ['C4..',46,0],
 ['C6..',69,0],
 ['Cp5.',-71,21],
 ['Cp3.',-50,28],
 ['Cp1.',-32,45],
 ['Cpz.',23,-90],
 ['Cp2.',32,-45],
 ['Cp4.',50,-28],
 ['Cp6.',71,-21],
 ['Fp1.',-92,-72],
 ['Fpz.',92,90],
 ['Fp2.',92,72],
 ['Af7.',-92,-52],
 ['Af3.',-74,-67],
 ['Afz.',69,90],
 ['Af4.',74,67],
 ['Af8.',92,52],
 ['F7..',-92,-36],
 ['F5..',-75,-41],
 ['F3..',-60,-51],
 ['F1..',-50,-68],
 ['Fz..',46,90],
 ['F2..',50,68],
 ['F4..',60,51],
 ['F6..',75,41],
 ['F8..',92,36],
 ['Ft7.',-92,-18],
 ['Ft8.',92,18],
 ['T7..',-92,0],
 ['T8..',92,0],
 ['T9..',-115,0],
 ['T10.',115,0],
 ['Tp7.',-92,18],
 ['Tp8.',92,-18],
 ['P7..',-92,36],
 ['P5..',-75,41],
 ['P3..',-60,51],
 ['P1..',-50,68],
 ['Pz..',46,-90],
 ['P2..',50,-68],
 ['P4..',60,-51],
 ['P6..',75,-41],
 ['P8..',92,-36],
 ['Po7.',-92,54],
 ['Po3.',-74,67],
 ['Poz.',69,-90],
 ['Po4.',74,-67],
 ['Po8.',92,-54],
 ['O1..',-92,72],
 ['Oz..',92,-90],
 ['O2..',92,-72],
 ['Iz..',115,-90]
 ]

#Versión de los ángulos en radianes.
anglesRad = []
piFactor = math.pi/180
for x in angles:
    anglesRad.append([x[0], x[1] * piFactor, x[2] * piFactor])
anglesRad

[['Fc5.', -1.239183768915974, -0.3665191429188092],
 ['Fc3.', -0.8726646259971648, -0.4886921905584123],
 ['Fc1.', -0.5585053606381855, -0.7853981633974483],
 ['Fcz.', 0.4014257279586958, 1.5707963267948966],
 ['Fc2.', 0.5585053606381855, 0.7853981633974483],
 ['Fc4.', 0.8726646259971648, 0.4886921905584123],
 ['Fc6.', 1.239183768915974, 0.3665191429188092],
 ['C5..', -1.2042771838760873, 0.0],
 ['C3..', -0.8028514559173916, 0.0],
 ['C1..', 0.4014257279586958, 0.0],
 ['Cz..', 0.0, 0.0],
 ['C2..', 0.4014257279586958, 0.0],
 ['C4..', 0.8028514559173916, 0.0],
 ['C6..', 1.2042771838760873, 0.0],
 ['Cp5.', -1.239183768915974, 0.3665191429188092],
 ['Cp3.', -0.8726646259971648, 0.4886921905584123],
 ['Cp1.', -0.5585053606381855, 0.7853981633974483],
 ['Cpz.', 0.4014257279586958, -1.5707963267948966],
 ['Cp2.', 0.5585053606381855, -0.7853981633974483],
 ['Cp4.', 0.8726646259971648, -0.4886921905584123],
 ['Cp6.', 1.239183768915974, -0.3665191429188092],
 ['Fp1.', -1.6057029118347832, -1.2566

In [5]:
nCubits = 4
nShots = 20000
strDevice = "qiskit.ibmq"
#strBackend="ibmq_belem"
#strBackend="ibmq_qasm_simulator"
strBackend="simulator_statevector"
strToken="7a60a88cb869db32476e0939ad7d1cecfeeff8d58659acccd3d7836f0a2979e6992786fe258164ec12c3217ca0b233781f65cce5e8ee4e9046c24b7e3b028768"
#dev = qml.device ("default.qubit", wires = nCubits)
dev = qml.device (strDevice, backend=strBackend, ibmqx_token=strToken, wires = nCubits, shots = nShots)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])


def EvalAsocLegendre4(theta):
  """
  Evaluar los Polinomios asociados de legendre en un ángulo dado utilizando un circutio de 4 cúbits. Esto generará todos los polinomios asociados de Legendre hasta l = 4.

  Parámetros: 
    theta: ángulo en radiantes sobre el cual calcular los polinomios asociados de Legendre.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "P_l_m" indicando el polinomio asociado de Legendre al que hace referencia y cuyo valor es el resultado de la evaluación de dicho polinómio sobre el ángulo dado.
  """
  nCubits = 4
  C = CircuitoCoseno(abs(theta), nCubits) #importante que el ángulo sea siempre positivo.
  S = CircuitoSeno(abs(theta), nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])

  #Polinomios de "m" negativo
  for l in range(1, nCubits + 1):
    for m in range(1, l+1):
      PolinomiosLegendre['P_' + str(l) + '_-' + str(m)] = ((-1)**(m) * (math.factorial(l-m)/math.factorial(l+m))) * PolinomiosLegendre['P_' + str(l) + '_' + str(m)]
  return PolinomiosLegendre

def EvalEsfericos4(theta, phi):
  """
  Realiza la evaluación de armónicos esféricos mediante cirucitos cuánticos de 4 cubits. Se obtiene el resultado de evalluar los armónicos esféricos para los ángulos dados hasta l = 4.

  Parámetros:
    theta: ángulo én radianes de la coordenada polar.
    phi: ángulo en radianes de la coordenada azimutal.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "Y_l_m" indicando el armónico esférico al que hace referencia y cuyo valor es el resultado de la evaluación de dicho armónico esféricos sobre los ángulos dados.
  """
  l = 4
  legendre = EvalAsocLegendre4(theta)
  AE = {}
  e = []
  for m in range(l+1):
    if (m==0):
      e.append((1, 0))
    else:
      e.append(ExpPhi(phi, (m)))
  for k in range(l+1):
    for m in range(0, k+1):
      coef = math.sqrt(((2*k + 1)*math.factorial(k - (m))) / (4*math.pi * math.factorial(k+(m)))) #Coeficiente de normalización
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(m)])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[m][0], legendre_normalizado * e[m][1])
      #lo mismo para "m" negativos
      coef = math.sqrt(((2*k + 1)*math.factorial(k + (m))) / (4*math.pi * math.factorial(k-(m))))
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(-m)])
      AE['Y_' + str(k) + '_' + str(-m)] = ( legendre_normalizado * e[m][0], legendre_normalizado * (-e[m][1]))

  return AE

Se calculan los armónicos esféricos utilizando circuitos cuánticos según la propuesta del trabajo. Para cada electrodo se guardan en un archivo los armónicos esféricos calculados. Si se interrumpe el proceso, se recupera la última ejecución según los datos del archivo.

In [6]:
anglesTmp = {}
for x in anglesRad:
    anglesTmp[x[0]] = [float(x[1]), float(x[2])]
resultadoAE = {}

for x in anglesTmp:
    xVal = anglesTmp[x]
    AE = EvalEsfericos4(xVal[1], xVal[0])
    calculadosAE = []
    for armonicos in AE.values():
        calculadosAE.append(complex(armonicos[0], armonicos[1]))
    resultadoAE[x] = calculadosAE
    
resultadoAE


{'Fc5.': [(0.28209479177387814+0j),
  (0.4584068766673195+0j),
  (-0.040334491636304655+0.11856809010423201j),
  (0.040334491636304655+0.11856809010423201j),
  (0.5122274411266179+0j),
  (-0.08461688203122748+0.24874150351227622j),
  (0.08461688203122748+0.24874150351227622j),
  (-0.039611332055599385-0.031097595633405972j),
  (-0.039611332055599385+0.031097595633405972j),
  (0.47035144959658143+0j),
  (-0.12602273766362965+0.3704589969601562j),
  (0.12602273766362962+0.37045899696015616j),
  (-0.09882566853574594-0.07758488591123285j),
  (-0.09882566853574594+0.07758488591123285j),
  (0.01680494116828128-0.010847189406478702j),
  (-0.01680494116828128-0.010847189406478703j),
  (0.3505204097035295+0j),
  (-0.1594014127828858+0.4685796276795382j),
  (0.1594014127828858+0.46857962767953826j),
  (-0.19544198167878588-0.1534352772461051j),
  (-0.1954419816787859+0.15343527724610512j),
  (0.0472991874122445-0.030530499303474963j),
  (-0.0472991874122445-0.030530499303474963j),
  (0.00254949

Se calculan los armónicos esféricos mediante la función correspondiente de la librería "scipy" de python. Hay que tener en cuenta que para esta función "theta" es la coordenada azimutal y "phi" la polar, al revés de lo considerado anteriormente.

In [7]:
nCubits = 4
m = []
v = []
for l in range(nCubits + 1):
    for n in range(0, l+1):
        m.append(n)
        if (n>0):
            m.append(-n)
            v.append(l)
        v.append(l)
resultadoSph = {}
for x in anglesRad:
    resultadoSph[x[0]]= sph_harm(m, v, x[1], x[2])
resultadoSph

{'Fc5.': array([ 0.28209479+0.j        ,  0.45614974+0.j        ,
        -0.04030991+0.11706847j,  0.04030991+0.11706847j,
         0.5092682 +0.j        , -0.08414891+0.24438619j,
         0.08414891+0.24438619j, -0.03909185-0.0305419j ,
        -0.03909185+0.0305419j ,  0.47306573+0.j        ,
        -0.1266131 +0.36771115j,  0.1266131 +0.36771115j,
        -0.0965577 -0.07543915j, -0.0965577 +0.07543915j,
         0.01610453-0.0104584j , -0.01610453-0.0104584j ,
         0.3639207 +0.j        , -0.15979636+0.46408233j,
         0.15979636+0.46408233j, -0.1726922 -0.13492193j,
        -0.1726922 +0.13492193j,  0.04510461-0.02929128j,
        -0.04510461-0.02929128j,  0.00176578+0.00708217j,
         0.00176578-0.00708217j]),
 'Fc3.': array([ 0.28209479+0.j        ,  0.43141041+0.j        ,
        -0.10425994+0.12425216j,  0.10425994+0.12425216j,
         0.42224287+0.j        , -0.20584362+0.24531488j,
         0.20584362+0.24531488j, -0.01478375-0.08384279j,
        -0.01478375+0

Se comparan los resultados para obtener el error.

In [8]:
error = {}
for x in resultadoAE:
    error[x] = (np.absolute(resultadoAE[x]) - np.absolute(resultadoSph[x]))
error

{'Fc5.': tensor([ 0.        ,  0.00225714,  0.00142679,  0.00142679,  0.00295924,
          0.00427219,  0.00427219,  0.00075162,  0.00075162, -0.00271428,
          0.00240863,  0.00240863,  0.00310858,  0.00310858,  0.00079924,
          0.00079924, -0.01340029,  0.00412711,  0.00412711,  0.02932571,
          0.02932571,  0.00251567,  0.00251567,  0.00316615,  0.00316615], requires_grad=True),
 'Fc3.': tensor([ 0.00000000e+00,  1.17613517e-06,  2.09975694e-03,
          2.09975694e-03, -1.46321350e-03,  4.14649923e-03,
          4.14649923e-03, -5.32315200e-04, -5.32315200e-04,
         -2.22833803e-03,  3.26710982e-03,  3.26710982e-03,
          1.79457186e-03,  1.79457186e-03,  1.27094931e-03,
          1.27094931e-03,  8.16363859e-03,  1.85131088e-03,
          1.85131088e-03,  3.53691767e-02,  3.53691767e-02,
          3.36686583e-03,  3.36686583e-03,  7.55255608e-03,
          7.55255608e-03], requires_grad=True),
 'Fc1.': tensor([ 0.        ,  0.00209768,  0.00017978,  0.00017

Error máximo:

In [9]:
error = {}
for x in resultadoAE:
    error[x] = max((np.absolute(resultadoAE[x]) - np.absolute(resultadoSph[x])))
error

{'Fc5.': tensor(0.02932571, requires_grad=True),
 'Fc3.': tensor(0.03536918, requires_grad=True),
 'Fc1.': tensor(0.00714706, requires_grad=True),
 'Fcz.': tensor(0.00701572, requires_grad=True),
 'Fc2.': tensor(0.00246814, requires_grad=True),
 'Fc4.': tensor(0.03656706, requires_grad=True),
 'Fc6.': tensor(0.01854695, requires_grad=True),
 'C5..': tensor(0.0173318, requires_grad=True),
 'C3..': tensor(0.0143904, requires_grad=True),
 'C1..': tensor(0.08707643, requires_grad=True),
 'Cz..': tensor(0.00822935, requires_grad=True),
 'C2..': tensor(0.03074373, requires_grad=True),
 'C4..': tensor(0.01102458, requires_grad=True),
 'C6..': tensor(0.03578142, requires_grad=True),
 'Cp5.': tensor(0.01783289, requires_grad=True),
 'Cp3.': tensor(0.04370633, requires_grad=True),
 'Cp1.': tensor(0.02885301, requires_grad=True),
 'Cpz.': tensor(0.03255271, requires_grad=True),
 'Cp2.': tensor(0.0647391, requires_grad=True),
 'Cp4.': tensor(0.03840343, requires_grad=True),
 'Cp6.': tensor(0.04266

Como se puede observar el máximo error es del orden de $10^{-16}$. Dicho error resulta despreciable y viene dado por la limitación en la precisión de los cálculos realizados mediante la librería "scipy".

In [10]:
tolerancia = 10**(-15)
print("Electrodos con error superior a " + str(tolerancia) + ":")
print("-----------------------------------------")
noError = True
for x in error:
    if abs(error[x]) > tolerancia:
        print(x + "-->" + str(error[x]))
        noError = False
if noError:
    print("No hay errores superiores a " + str(tolerancia) + ".")
print("-----------------------------------------")

Electrodos con error superior a 1e-15:
-----------------------------------------
Fc5.-->0.029325712276036586
Fc3.-->0.03536917672173212
Fc1.-->0.007147059335610481
Fcz.-->0.00701571505269412
Fc2.-->0.002468143299788589
Fc4.-->0.036567057938611716
Fc6.-->0.018546947225977328
C5..-->0.017331801221780355
C3..-->0.014390404948974773
C1..-->0.08707643496906453
Cz..-->0.008229352144726854
C2..-->0.03074372655746782
C4..-->0.011024576654062411
C6..-->0.035781422570577925
Cp5.-->0.01783289478429967
Cp3.-->0.04370632806376784
Cp1.-->0.028853007921122054
Cpz.-->0.03255271385199321
Cp2.-->0.06473910299961244
Cp4.-->0.038403434414818194
Cp6.-->0.04266605192264389
Fp1.-->0.0029693688232330373
Fpz.-->0.00757824372531235
Fp2.-->0.027649870382020963
Af7.-->0.011330837788956188
Af3.-->0.012246822769620917
Afz.-->0.06234858827878015
Af4.-->0.04080221261205717
Af8.-->0.006174289239992636
F7..-->0.050613059507532476
F5..-->0.015477927778451195
F3..-->0.07619362001695568
F1..-->0.04344318146157336
Fz..-->0